# Tranform .gif to binary .txt files

In [ ]:
import os
import numpy as np
from PIL import Image

input_dir = "." # Folder with .gif files
output_dir = "txt_binary" # Output file

threshold = 128 # Grayscale threshold

os.makedirs(output_dir, exist_ok=True)

# Process each .gif
for file in os.listdir(input_dir):
    if file.lower().endswith(".gif"):
        gif_path = os.path.join(input_dir, file)
        name = os.path.splitext(file)[0]
        txt_path = os.path.join(output_dir, name + ".txt")

        # Load .gif
        img = Image.open(gif_path).convert("L")  # Grayscale

        # Convert to NumPy array
        arr = np.array(img)

        # To binary (if 1 = object, 0 = background)
        binary = (arr >= threshold).astype(np.uint8)

        # Save as .txt
        with open(txt_path, "w") as f:
            for row in binary:
                f.write("".join(map(str, row)) + "\n")

        print(f"{file}: saved {binary.shape} -> {name}.txt")

apple-17.gif: saved (256, 256) -> apple-17.txt
apple-18.gif: saved (256, 256) -> apple-18.txt
bat-13.gif: saved (472, 503) -> bat-13.txt
bird-19.gif: saved (252, 320) -> bird-19.txt
device2-14.gif: saved (512, 512) -> device2-14.txt
hammer-4.gif: saved (256, 256) -> hammer-4.txt
pencil-15.gif: saved (359, 250) -> pencil-15.txt
pencil-19.gif: saved (358, 239) -> pencil-19.txt
ray-14.gif: saved (595, 585) -> ray-14.txt
turtle-4.gif: saved (186, 333) -> turtle-4.txt


# Area count

In [ ]:
txt_dir = "txt_binary"
output_summary = "area_counts.txt"

area_results = []

# Work with binaries
for file in sorted(os.listdir(txt_dir)):
    if file.endswith(".txt"):
        path = os.path.join(txt_dir, file)

        # Load binary
        binary = np.array(
            [[int(c) for c in line.strip()]
             for line in open(path)],
            dtype=np.uint8
        )

        area = int(binary.sum())
        area_results.append((file, area))

        print(f"{file}: area = {area}")

# Summary
with open(output_summary, "w") as f:
    for name, area in area_results:
        f.write(f"{name}\t{area}\n")

print(f"\nArea saved in {output_summary}")

apple-17.txt: area = 37498
apple-18.txt: area = 34893
bat-13.txt: area = 43481
bird-19.txt: area = 20797
device2-14.txt: area = 87064
hammer-4.txt: area = 7790
pencil-15.txt: area = 7868
pencil-19.txt: area = 8102
ray-14.txt: area = 68201
turtle-4.txt: area = 36013

Area summary saved to area_counts.txt


# Get N4 and N8 connected components

In [ ]:
txt_dir = "txt_binary"

def load_binary_txt(path):
    return np.array(
        [[int(c) for c in line.strip()]
         for line in open(path)],
        dtype=np.uint8
    )

# Union and Find
class UnionFind:
    def __init__(self):
        self.parent = {}

    def find(self, x):
        if self.parent.setdefault(x, x) != x:
            self.parent[x] = self.find(self.parent[x])
        return self.parent[x]

    def union(self, a, b):
        self.parent[self.find(b)] = self.find(a)

# Get Connected Components
def connected_components(binary, neighborhood="N4"):
    h, w = binary.shape
    labels = np.zeros((h, w), dtype=int)
    uf = UnionFind()
    current_label = 1

    for i in range(h):
        for j in range(w):
            if binary[i, j] == 0:
                continue

            neighbors = []

            # N4 neighbors
            if j > 0 and labels[i, j-1] > 0:
                neighbors.append(labels[i, j-1])
            if i > 0 and labels[i-1, j] > 0:
                neighbors.append(labels[i-1, j])

            # N8 neighbors
            if neighborhood == "N8" and i > 0:
                if j > 0 and labels[i-1, j-1] > 0:
                    neighbors.append(labels[i-1, j-1])
                if j < w-1 and labels[i-1, j+1] > 0:
                    neighbors.append(labels[i-1, j+1])

            if not neighbors:
                labels[i, j] = current_label
                current_label += 1
            else:
                min_label = min(neighbors)
                labels[i, j] = min_label
                for lbl in neighbors:
                    uf.union(min_label, lbl)

    # Second pass to resolve labels
    label_map = {}
    new_label = 1

    for i in range(h):
        for j in range(w):
            if labels[i, j] > 0:
                root = uf.find(labels[i, j])
                if root not in label_map:
                    label_map[root] = new_label
                    new_label += 1
                labels[i, j] = label_map[root]

    return labels, new_label - 1

# Main
print("Connected Components:\n")

for file in sorted(os.listdir(txt_dir)):
    if file.endswith(".txt"):
        binary = load_binary_txt(os.path.join(txt_dir, file))

        _, n4_count = connected_components(binary, "N4")
        _, n8_count = connected_components(binary, "N8")

        print(f"{file}")
        print(f"  N4 components: {n4_count}")
        print(f"  N8 components: {n8_count}\n")

Connected Components Summary

apple-17.txt
  N4 components: 1
  N8 components: 1

apple-18.txt
  N4 components: 1
  N8 components: 1

bat-13.txt
  N4 components: 1
  N8 components: 1

bird-19.txt
  N4 components: 2
  N8 components: 1

device2-14.txt
  N4 components: 1
  N8 components: 1

hammer-4.txt
  N4 components: 1
  N8 components: 1

pencil-15.txt
  N4 components: 1
  N8 components: 1

pencil-19.txt
  N4 components: 1
  N8 components: 1

ray-14.txt
  N4 components: 1
  N8 components: 1

turtle-4.txt
  N4 components: 1
  N8 components: 1



# (OPTIONAL) Save components as images

In [15]:
import os
import numpy as np
import matplotlib.pyplot as plt

txt_dir = "txt_binary"
out_dir = "components_visualization"
os.makedirs(out_dir, exist_ok=True)

# Load binaries
def load_binary_txt(path):
    return np.array(
        [[int(c) for c in line.strip()]
         for line in open(path)],
        dtype=np.uint8
    )

# Same algorithm
class UnionFind:
    def __init__(self):
        self.parent = {}

    def find(self, x):
        if self.parent.setdefault(x, x) != x:
            self.parent[x] = self.find(self.parent[x])
        return self.parent[x]

    def union(self, a, b):
        self.parent[self.find(b)] = self.find(a)

def connected_components(binary, neighborhood="N4"):
    h, w = binary.shape
    labels = np.zeros((h, w), dtype=int)
    uf = UnionFind()
    current_label = 1

    for i in range(h):
        for j in range(w):
            if binary[i, j] == 0:
                continue

            neighbors = []

            if j > 0 and labels[i, j-1] > 0:
                neighbors.append(labels[i, j-1])
            if i > 0 and labels[i-1, j] > 0:
                neighbors.append(labels[i-1, j])

            if neighborhood == "N8" and i > 0:
                if j > 0 and labels[i-1, j-1] > 0:
                    neighbors.append(labels[i-1, j-1])
                if j < w-1 and labels[i-1, j+1] > 0:
                    neighbors.append(labels[i-1, j+1])

            if not neighbors:
                labels[i, j] = current_label
                current_label += 1
            else:
                min_label = min(neighbors)
                labels[i, j] = min_label
                for lbl in neighbors:
                    uf.union(min_label, lbl)

    label_map = {}
    new_label = 1

    for i in range(h):
        for j in range(w):
            if labels[i, j] > 0:
                root = uf.find(labels[i, j])
                if root not in label_map:
                    label_map[root] = new_label
                    new_label += 1
                labels[i, j] = label_map[root]

    return labels, new_label - 1

# Save result as images
def save_labeled_image(labels, output_path, title):
    plt.figure(figsize=(6, 6))
    plt.imshow(labels, cmap="tab20")
    plt.axis("off")
    plt.title(title)
    plt.savefig(output_path, bbox_inches="tight", pad_inches=0)
    plt.close()

# Main
for file in sorted(os.listdir(txt_dir)):
    if file.endswith(".txt"):
        binary = load_binary_txt(os.path.join(txt_dir, file))

        labels_n4, n4_count = connected_components(binary, "N4")
        labels_n8, n8_count = connected_components(binary, "N8")

        base = os.path.splitext(file)[0]

        save_labeled_image(
            labels_n4,
            os.path.join(out_dir, f"{base}_N4_components.png"),
            f"{base} — N4 ({n4_count} components)"
        )

        save_labeled_image(
            labels_n8,
            os.path.join(out_dir, f"{base}_N8_components.png"),
            f"{base} — N8 ({n8_count} components)"
        )

print("All labeled component images saved...")

Labeled component images saved successfully.


# N4 Outlines Count

In [18]:
import os
import numpy as np
import matplotlib.pyplot as plt

# Directories
txt_dir = "txt_binary"
out_dir = "n4_outlines"
os.makedirs(out_dir, exist_ok=True)

# Load binary
def load_binary_txt(path):
    return np.array(
        [[int(c) for c in line.strip()]
         for line in open(path)],
        dtype=np.uint8
    )

# N4 Outline Extraction
def extract_n4_outline(binary):
    h, w = binary.shape
    outline = np.zeros_like(binary, dtype=np.uint8)

    for i in range(h):
        for j in range(w):
            if binary[i, j] == 0:
                continue

            # Check N4 neighbors
            neighbors = [
                (i-1, j),
                (i+1, j),
                (i, j-1),
                (i, j+1)
            ]

            for ni, nj in neighbors:
                if ni < 0 or ni >= h or nj < 0 or nj >= w:
                    outline[i, j] = 1
                    break
                if binary[ni, nj] == 0:
                    outline[i, j] = 1
                    break

    return outline

# Save outline image
def save_outline_image(outline, output_path):
    plt.figure(figsize=(outline.shape[1]/100, outline.shape[0]/100), dpi=100)
    plt.imshow(outline, cmap="gray")
    plt.axis("off")
    plt.savefig(output_path, bbox_inches="tight", pad_inches=0)
    plt.close()

# Main
print("N4 Outline Pixel Counts\n")

for file in sorted(os.listdir(txt_dir)):
    if file.endswith(".txt"):
        binary = load_binary_txt(os.path.join(txt_dir, file))
        outline = extract_n4_outline(binary)

        count = int(outline.sum())
        base = os.path.splitext(file)[0]

        print(f"{file}: N4 outline pixels = {count}")

        save_outline_image(
            outline,
            os.path.join(out_dir, f"{base}_N4_outline.png")
        )

print("\nAll N4 outline images saved...")

N4 Outline Pixel Counts

apple-17.txt: N4 outline pixels = 774
apple-18.txt: N4 outline pixels = 778
bat-13.txt: N4 outline pixels = 1278
bird-19.txt: N4 outline pixels = 838
device2-14.txt: N4 outline pixels = 1460
hammer-4.txt: N4 outline pixels = 611
pencil-15.txt: N4 outline pixels = 468
pencil-19.txt: N4 outline pixels = 561
ray-14.txt: N4 outline pixels = 1754
turtle-4.txt: N4 outline pixels = 980

All N4 outline images saved successfully.
